# Простая линейная регрессия (Simple Regression)

---

**Простой называют линейную регрессию, в модели которой число независимых переменных равно 1.**

---

$y=b_{0} + b_{1}x$

$y$ - зависимая переменная, что мы измеряем в эксперименте (dependent variable)

$x$ - независимая переменная (independent variable)

$b_{0}$ - показывает, чему равна зависимая переменная, если предиктор равен нулю (intercept)

$b_{1}$ - показывает, насколько изменяется зависимая переменная при изменении независимой переменной (coef | slope)

---

Параметр модели $b_{1}$ вычисляется при помощи метода наименьших квадратов (Ordinary Least Squares).

---

<center><img src="images/linear_regression_marks.png" alt="linear_regression_marks" width=500 height=500"/></center>

![linear_regression.gif](images/linear_regression.gif)

## Импорт пакетов
- `matplotlib.pyplot` и `pylab` - рисование графиков
- `pandas` - работа с таблицами
- `numpy` - работа с матрицами

In [ ]:
import matplotlib.pyplot as plt
import pylab as pl

import pandas as pd

import numpy as np

%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"]=10,10

## Анализ данных

`FuelConsumption.csv` (Рейтинги расхода топлива):

[Dataset source](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

-   **MODELYEAR** e.g. 2014
-   **MAKE** e.g. Acura
-   **MODEL** e.g. ILX
-   **VEHICLE CLASS** e.g. SUV
-   **ENGINE SIZE** e.g. 4.7
-   **CYLINDERS** e.g 6
-   **TRANSMISSION** e.g. A6
-   **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
-   **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
-   **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
-   **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0


### Загрузка данных

In [ ]:
import os
notebook_path = os.path.abspath("Notebook.ipynb")    # получить "точку отсчета" для поиска файла
data_csv = os.path.join(os.path.dirname(notebook_path), "data/FuelConsumptionCo2.csv")
df = pd.read_csv(data_csv)

# вывести первые 5 строк из таблицы
df.head()

In [ ]:
# вывести последние 5 строк таблицы
df.tail()

In [ ]:
df.info()

### Исследование данных (Data Exploration)

**По умолчанию будет выдана информация только для количественных признаков.**
- `count` - количество элементов
- `mean` - среднее значение
- `std` - стандартное отклонение
- `min` - минимальное значение
- `25%` - нижний квартиль
- `50%` - медиана
- `75%` - верхний квартиль
- `max` - максимальное значение

In [ ]:
# обзор Dataframe из файла FuelConsumptionCo2.csv
df.describe()

In [ ]:
# выбрать несколько столбцов для дальнейшего рассмотрения
short_df = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]

short_df.head(9)    # вывести первые 9 строк

In [ ]:
# выбрать несколько столбцов для визуализации
viz = short_df[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]

print('type: ', type(viz))    # вывести тип данных переменной viz

viz.hist()    # гистограмма
plt.show()

In [ ]:
plt.scatter(short_df.FUELCONSUMPTION_COMB,
            short_df.CO2EMISSIONS,
            color='green')
plt.xlabel("FUELCONSUMPTION_COMB", fontsize=20)
plt.ylabel("Emission", fontsize=20)
plt.show()

In [ ]:
plt.scatter(short_df.ENGINESIZE,
            short_df.CO2EMISSIONS,
            color='red')
plt.xlabel("Engine size", fontsize=20)
plt.ylabel("Emission", fontsize=20)
plt.show()

In [ ]:
plt.scatter(short_df.CYLINDERS,
            short_df.CO2EMISSIONS,
            color='chocolate')
plt.xlabel("Cylinders", fontsize=20)
plt.ylabel("Emission", fontsize=20)
plt.show()

## Обучение модели (Train) - Classic

In [ ]:
# разделить данные на train (80%) и test (20%)
msk = np.random.rand(len(df)) < 0.8
train = short_df[msk]
test = short_df[~msk]

In [ ]:
# импортировать из пакета с моделями модель линейной регрессии
from sklearn import linear_model

# создать объект линейной регрессии
simple_regr = linear_model.LinearRegression()

# разделить независимую и зависимую переменные
X_train = np.asanyarray(train[['ENGINESIZE']])
y_train = np.asanyarray(train[['CO2EMISSIONS']])

# обучить модель
simple_regr.fit(X_train, y_train)

# вывести результаты обучения
print('Coefficients: ', simple_regr.coef_)
print('Intercept: ',simple_regr.intercept_)

## Визуализация результата обучения модели

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='red')
plt.plot(X_train, simple_regr.coef_[0][0]*X_train + simple_regr.intercept_[0], '-g', linewidth=6)
plt.xlabel("Engine size", fontsize=20)
plt.ylabel("Emission", fontsize=20)

## Оценка качества модели (Evaluation)

In [ ]:
# использовать обученную модель для предсказания на test выборке
y_predicted = simple_regr.predict(test[['ENGINESIZE']])
X_test = np.asanyarray(test[['ENGINESIZE']])
y_test = np.asanyarray(test[['CO2EMISSIONS']])

# вывести результаты предсказания
print(f'Variance score: {simple_regr.score(X_test, y_test)}')    # Coefficient of determination R^2 of the prediction
print(f"Residual sum of squares: {np.mean((y_predicted - y_test) ** 2)}")    # MSE

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, y_predicted)    # эквивалентно simple_regr.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error

# MSE
mean_squared_error(y_test, y_predicted, squared=True)   # эквивалентно np.mean((y_predicted - y_test) ** 2)

---

## Обучение модели (Train) - XGBoost for Regression

In [ ]:
import xgboost as xg

xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)

# Fitting the model
xgb_r.fit(X_train, y_train)

In [ ]:
# Predict the model
y_predicted = xgb_r.predict(X_test)

# вывести результаты оценки предсказания
print(f'R2: {r2_score(y_test, y_predicted)}')
print(f"MSE: {mean_squared_error(y_test, y_predicted, squared=True)}")
print(f"MAE: {mean_squared_error(y_test, y_predicted, squared=False)}")
